In [1]:
import math 
import numpy
import torch 
print(torch.cuda.is_available())
device = torch.device("cuda:0")
# A True is gonna be printed
#dtype = torch.float32

True


#####This cell is copied from internet.
#####But I better doc is here https://pytorch.org/docs/stable/notes/extending.html

class RoundNoGradient(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        print(F"forward {x}")
        return x.round()
    @staticmethod
    def backward(ctx, g):
        print(F"backward {g}")
        return g 


l = torch.nn.L1Loss()
input = torch.autograd.Variable(torch.randn(3), requires_grad=True)
x = RoundNoGradient.apply(input)
print(F"x {x}")
y = x.mean()
#x = input.round()
y.backward()

#output = l(x, torch.autograd.Variable(torch.randn(3)))
#output.backward()
print(input.grad)

In [33]:
class SafeMul(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, y):
        with torch.no_grad():
            r = (x*y).numpy()
        #-1
        temp = r*0
        temp2 = temp != 0.
        ctx.save_for_backward(x, y, torch.tensor(temp2))
        
        for i, f in enumerate(temp2):
            if f:
                r[i] = 0
        #-2
        return torch.tensor(r, requires_grad = True)
    
    @staticmethod
    def backward(ctx, g):
        x, y, flags = ctx.saved_tensors
        #print(F"Inside backward g {g}")
        #print(F"Inside backward x {x}")
        #print(F"Inside backward y {y}")
        #print(F"flags {flags}")
        
        for i, f in enumerate(flags):
            #print(i,f)
            if f:
                #print(i,f,123)
                g[(i)] = 0.
        #-2
        temp = x*0
        temp2 = temp != 0.
        for i, f in enumerate(temp2):
            if f:
                x[i] = 0.
        #-2
        temp = y*0
        temp2 = temp != 0.
        for i, f in enumerate(temp2):
            if f:
                y[i] = 0.
        #-2
        #print(F"Inside backward x {x}")
        #print(F"Inside backward y {y}")
        #print(F"Inside backward g {g}")
        return g*y, g*x

    
x = torch.tensor([3., float("nan"), 100e100], requires_grad = True)
#print(F"x {x}")
#x = torch.tensor([1., 1., 1.], requires_grad = True)
#print(x)
y = torch.tensor([11., 111., 1111.], requires_grad = True)
#print(F"y {y}")
r = SafeMul.apply(x, y).mean()
#r = layer.forward(x, y).mean()

r.backward()
print(x.grad)
print(y.grad)


tensor([3.6667, 0.0000, 0.0000])
tensor([1., 0., 0.])


C:\Users\YAGAOD~1\AppData\Local\Temp/ipykernel_12016/3980839213.py:7: RuntimeWarning: invalid value encountered in multiply
  temp = r*0


In [7]:
a = torch.tensor([1,2,3])#.numpy()
a[1] = 4
a

tensor([1, 4, 3])

In [13]:
'''This part is actually something I made for myself in real pytorch projects.
This one shows how to implement stateful, fully customized backward propagation.
Notice, the first class inherits from torch.autograd.Function
Also Notice, how the backward function in the first class returns 5 values.
But this example is probably not perfect. "with torch.no_grad():" could probably help.
Idk, I may try this detail in the future.
'''
import torch

class LLC2_functional_part(torch.autograd.Function):
    '''Linear leaky clamp 2
    The forward and backward hebavior differently.
    '''
    @staticmethod
    def forward(ctx, x, a, b, factor, negative_slope):
        #assert factor.item()>=1.
        #assert negative_slope.item()>0. and negative_slope.item()<1.
        ctx.save_for_backward(x, a, b, torch.tensor([factor]), torch.tensor([negative_slope]))
        x = torch.abs(x)
        x = a * x + b
        # Then, double relu to make the result almost inside 0 to 1
        x = torch.relu(x)
        x = 1 - torch.relu(1 - x)
        return x
        pass
    @staticmethod
    def backward(ctx, g):
        x, a, b, factor, negative_slope = ctx.saved_tensors
        factor = factor.item()
        negative_slope = negative_slope.item()
        boundary = torch.abs(b/a)*factor
        x_abs = torch.abs(x)
        out_flags = x_abs>boundary
        ori_ax_b = a * torch.abs(x) + b
        in_flags = (ori_ax_b < 1.) * (ori_ax_b > 0.)
        flags = in_flags + negative_slope * out_flags
        b_grad = g*flags
        a_grad = g*flags*x_abs
        x_less_than_0 = x<0.
        x_less_than_0 = x_less_than_0 * -2 + 1
        x_grad = g*flags*a*x_less_than_0
        return x_grad, a_grad, b_grad, None, None
        pass
    def __str__(self):
        return F'LLC2_functional. It\'s designed only for the LLC2 layer. It behaviors differently in forward propagation and backward propagation.'
        pass
    pass
class LLC2(torch.nn.Module):
    '''Linear leaky clamp version 2.0
    The forward pass is pretty simple.
    '''
    def __init__(self, size, * , a = -1., b = 2., boundary_factor = 2., negative_slope=0.01):
        super(LLC2, self).__init__()
        self.a = torch.nn.Parameter(torch.full((size, ), a, dtype = torch.float32), True)
        self.b = torch.nn.Parameter(torch.full((size, ), b, dtype = torch.float32), True)
        assert boundary_factor>=1.
        self.boundary_factor = boundary_factor
        self.negative_slope = negative_slope
        pass
    def forward(self, x):
        self.a = torch.nn.Parameter(-torch.abs(self.a))
        x = LLC2_functional_part.apply(x, self.a, self.b, self.boundary_factor, self.negative_slope)
        # The parameters for the specific function are: x, a, b, factor, negative_slope
        return x
        pass
    def __str__(self):
        return F'Linear leaky clamp ver 2.0. a: {self.a}  b: {self.b}  the boundary: {torch.abs(self.b/self.a)} boundary_factor: {self.boundary_factor:.1f}  negative_slope: {self.negative_slope:.4f}'
    pass

input = torch.tensor([0.9,1.1,1.9,2.1,3.9,4.1], requires_grad = True)
input = torch.cat((input, input*-1))
#input = input.view(1, -1) # Uncomment this line to check if this layer works for (batch_size, dimention).
print(F"input:{input}")
l = LLC2(input.shape[0])
output = l(input)
print(F"output before:{output}")
output = output.mean()*3
#output = output/output.item()
output.backward()
print(F"output after:{output}")
print(F"grad of a: {l.a.grad}")
print(F"grad of b: {l.b.grad}")
print(F"grad of input: {input.grad}")


input:tensor([ 0.9000,  1.1000,  1.9000,  2.1000,  3.9000,  4.1000, -0.9000, -1.1000,
        -1.9000, -2.1000, -3.9000, -4.1000], grad_fn=<CatBackward0>)
output before:tensor([1.0000, 0.9000, 0.1000, 0.0000, 0.0000, 0.0000, 1.0000, 0.9000, 0.1000,
        0.0000, 0.0000, 0.0000], grad_fn=<LLC2_functional_partBackward>)
output after:1.0
grad of a: tensor([0.0000, 0.2750, 0.4750, 0.0000, 0.0000, 0.0102, 0.0000, 0.2750, 0.4750,
        0.0000, 0.0000, 0.0102])
grad of b: tensor([0.0000, 0.2500, 0.2500, 0.0000, 0.0000, 0.0025, 0.0000, 0.2500, 0.2500,
        0.0000, 0.0000, 0.0025])
grad of input: None
